In [7]:
import csv
import re
import os
import numpy as np
import pandas as pd
import sys

In [16]:
sys.path.append('../..')

In [17]:
from utils import *

In [ ]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [ ]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/'

In [5]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [25]:
excel_spreadsheet = '/home/andrew/Downloads/WHOI_Asset_Tracking.xlsx'

In [163]:
NUTNR = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='NUTNR',series='B')

In [164]:
NUTNR

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
876,NUTNR,B,239,115084,A00065,CGINS-NUTNRB-00239,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00004\n3305-00108-00048\n3305-00108...,3305-00308-00001,3305-00508-00040,3305-00900-00075\n3305-00900-00144\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,GI01SUMO-00001\nCP04OSSM-00006,NaN,NSIF,Reading High nitrate levels during calibration...
877,NUTNR,B,240,115085,A00066,CGINS-NUTNRB-00240,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00003\n3305-00108-00029\n3305-00108...,NaN,NaN,3305-00900-00008\n3305-00900-00231\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,CP01CNSM-00001\nGS01SUMO-00002,NaN,NSIF,"09/2017: Clock issue - resets back to Jan 1, 2..."
878,NUTNR,B,260,115671,A00383,CGINS-NUTNRB-00260,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00010\n3305-00108-00056,NaN,3305-00508-00010,3305-00900-00109\n3305-00900-00317,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI Spare\nCP1 spare\nGS01SUMO-00001\nCP04OSSM-...,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
879,NUTNR,B,261,115672,A00384,CGINS-NUTNRB-00261,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00011\n3305-00108-00021\n3305-00108...,3305-00308-00007\n3305-00308-00031\n3305-00308...,3305-00508-00022\n3305-00508-00041,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,CP03ISSM-00002\nCP01CNSM-00005\nCP01CNSM-00007,NaN,NSIF,Returned to vendor 6/24/14 (RMA#2014-125)\nSen...
880,NUTNR,B,262,115673,A00385,CGINS-NUTNRB-00262,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00012\n3305-00108-00064,3305-00308-00002,NaN,3305-00900-00064\n3305-00900-00153\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI01SUMO-00001,NaN,NaN,Sent to vedor as part of trade in for new (SUN...
881,NUTNR,B,266,116564,A00880,CGINS-NUTNRB-00266,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00015\n3305-00108-00040\n3305-00108...,3305-00308-00004\n3305-00308-00032,3305-00508-00024,3305-00900-00036\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP04OSSM-00001\nCP04OSSM-00004\nCP01CNSM-00008,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
882,NUTNR,B,267,116562,A00878,CGINS-NUTNRB-00267,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00018\n3305-00108-00053,NaN,3305-00508-00009,3305-00900-00109\n3305-00900-00363,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,GS01SUMO-00001\nCP01CNSM-00006,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
883,NUTNR,B,268,116563,A00879,CGINS-NUTNRB-00268,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00017\n3305-00108-00046\n3305-00108...,3305-00308-00003\n3305-00308-00034,3305-00508-00023,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP03ISSM-00001\nCP03ISSM-00004\nCP03ISSM-00006,NaN,NSIF,4/2016: Scheduled to sample every half hour. S...
884,NUTNR,B,269,116565,A00881,CGINS-NUTNRB-00269,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00016\n3305-00108-00047\n3305-00114...,3305-00308-00005\n3305-00308-00009,3305-00508-00001,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP03ISSM-00005,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
885,NUTNR,B,270,116899,A01142,CGINS-NUTNRB-00270,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00020\n3305-00108-00050,3305-00308-00006\n3305-00308-00008,3305-00508-00038,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-09-24 00:00:00,CP3a Spare\nCP04OSSM-00002\nGI01SUMO-00003,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...


In [ ]:
uids = list(set(NUTNR['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    qct_series = NUTNR[NUTNR['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [ ]:
serial_nums = get_serial_nums(NUTNR, uids)
serial_nums

In [ ]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

In [ ]:
filename = cal_dict['CGINS-NUTNRB-01065'][1]
filename

In [ ]:
cal_directory

In [ ]:
filepath = generate_file_path(cal_directory,filename.split('.')[0],ext=['.zip'])
filepath

In [ ]:
with ZipFile(filepath) as zfile:
    filename = [name for name in zfile.namelist() if name.endswith('.cal') and name.startswith('SNA')]
    if len(filename) == 1:
        data = zfile.read(filename[0]).decode('ASCII')
    elif len(filename) > 1:
        raise Exception(f'More than one calibration file found in {filepath}.')
    else:
        FileExistsError(f"No .cal file found in {filepath}")

In [ ]:
data

In [ ]:
for line in data.splitlines():
    print(line.split(','))

In [ ]:
serial = None
for line in data.splitlines():
    if line.startswith('H'):
        # Get the serial number
        if 'SUNA' in line and serial == None:
            _, info = line.split(',')
            serial = info.split()[1]
        # 

In [ ]:
serial

# Check the most recent NUTNR Cals

CGINS-NUTNRB-01089__20190327.csv
CGINS-NUTNRB-01090__20190401.csv
CGINS-NUTNRB-01091__20190327.csv

Project_Files > Records > Instrument_Records > NUTNR > NUTNR_Results

In [4]:
os.listdir('/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/')

['3305-00308-00023.xls',
 '3305-00108-00001-A.log',
 '3305-00108-00001.docx',
 '3305-00108-00002-A.log',
 '3305-00108-00002.docx',
 '3305-00108-00003-A.txt',
 '3305-00108-00003.docx',
 '3305-00108-00004-A.txt',
 '3305-00108-00004.docx',
 '3305-00108-00005-A.txt',
 '3305-00108-00005.doc',
 '3305-00108-00006-A.txt',
 '3305-00108-00006.doc',
 '3305-00108-00007-A.txt',
 '3305-00108-00007.doc',
 '3305-00108-00008-A.txt',
 '3305-00108-00009-A',
 '3305-00108-00009.doc',
 '3305-00108-00010-A.txt',
 '3305-00108-00010.docx',
 '3305-00108-00011-A.txt',
 '3305-00108-00011.docx',
 '3305-00108-00012-A.txt',
 '3305-00108-00012.docx',
 '3305-00108-00013-A.txt',
 '3305-00108-00013.doc',
 '3305-00108-00014-A.txt',
 '3305-00108-00014.doc',
 '3305-00108-00015-A.TXT',
 '3305-00108-00015.docx',
 '3305-00108-00016-A.TXT',
 '3305-00108-00017-A.TXT',
 '3305-00108-00017.docx',
 '3305-00108-00018-A.TXT',
 '3305-00108-00018.docx',
 '3305-00108-00019-A.log',
 '3305-00108-00019.docx',
 '3305-00108-00020-A.log',
 '3

In [166]:
NUTNR[NUTNR['Supplier\nSerial Number'] == 'NTR-1091']

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
925,NUTNR,B,NTR-1091,118559,A02242,CGINS-NUTNRB-01091,SUNA,1336-00014-10002,2.5.1,Satlantic,...,3305-00127-00017,3305-00327-00056,NaN,NaN,NaN,2017-11-17 00:00:00,NaN,CP03ISSM-00010,NSIF,NaN


In [171]:
filepath = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results'

In [172]:
from zipfile import ZipFile

In [227]:
for file in os.listdir(filepath):
    if '3305-00327-00054' in file and file.endswith('.zip'):
        print(file)
        zipfile = file

3305-00327-00054-A.zip


In [228]:
zfile = ZipFile(filepath+'/'+zipfile)

In [229]:
zfile.namelist()

['3305-00327-00054/NTR-1090-SelfTest-2019-04-01.txt',
 '3305-00327-00054/SNA1090B.CAL',
 '3305-00327-00054/SNA1090C.CAL',
 '3305-00327-00054/SNA1090C.pdf',
 '3305-00327-00054/SUNA-1090-Summary-20190401T201713.pdf']

In [231]:
data = zfile.read(zfile.namelist()[2]).decode('ASCII')

In [232]:
data

'H,SUNA 1090 Cal B  extinction coefficients and reference spectra\r\nH,File generated by SUNACom 1.2.5\r\nH,File format version 3\r\nH,File creation time 01-Apr-2019 20:09:37\r\nH,File generated by Internal Software Suite 1.9.21_243\r\nH,File format version 3\r\nH,File creation time 24-Oct-2017 11:16:11\r\nH,Operator TH\r\nH,PATH_LENGTH 10\r\nH,INT_PERIOD 300\r\nH,CONC_CAL_NO3 40.00\r\nH,CONC_CAL_SWA 34.92\r\nH,T_S_CORRECTABLE\r\nH,T_CAL 19.95\r\nH,T_CAL_SWA 19.95\r\nH,NitrateFile SUNA_1090_001.xml\r\nH,DIW Log file SUNA1090_Cal01_DIW_3.raw\r\nH,LNSW Log file SUNA1090_Cal01_LNSW_1.raw\r\nH,Nitrate in LNSW Log file SUNA1090_Cal01_NO3_1.raw\r\nH,Wavelength,nm\r\nH,NITRATE,uM\r\nH,AUX1,none\r\nH,AUX2,none\r\nH,Reference,counts\r\nH,Wavelength,NO3,SWA,TSWA,Reference\r\nE,189.32,0.00208249,0.00263663,0.00292439,-12.66666667\r\nE,190.12,0.00278842,0.00220094,0.00239400,-10.58333333\r\nE,190.92,-0.00420238,0.00905437,0.00965837,-10.62500000\r\nE,191.72,0.00180065,0.00157827,0.00165103,-8.2916

In [233]:
print(data)

H,SUNA 1090 Cal B  extinction coefficients and reference spectra
H,File generated by SUNACom 1.2.5
H,File format version 3
H,File creation time 01-Apr-2019 20:09:37
H,File generated by Internal Software Suite 1.9.21_243
H,File format version 3
H,File creation time 24-Oct-2017 11:16:11
H,Operator TH
H,PATH_LENGTH 10
H,INT_PERIOD 300
H,CONC_CAL_NO3 40.00
H,CONC_CAL_SWA 34.92
H,T_S_CORRECTABLE
H,T_CAL 19.95
H,T_CAL_SWA 19.95
H,NitrateFile SUNA_1090_001.xml
H,DIW Log file SUNA1090_Cal01_DIW_3.raw
H,LNSW Log file SUNA1090_Cal01_LNSW_1.raw
H,Nitrate in LNSW Log file SUNA1090_Cal01_NO3_1.raw
H,Wavelength,nm
H,NITRATE,uM
H,AUX1,none
H,AUX2,none
H,Reference,counts
H,Wavelength,NO3,SWA,TSWA,Reference
E,189.32,0.00208249,0.00263663,0.00292439,-12.66666667
E,190.12,0.00278842,0.00220094,0.00239400,-10.58333333
E,190.92,-0.00420238,0.00905437,0.00965837,-10.62500000
E,191.72,0.00180065,0.00157827,0.00165103,-8.29166667
E,192.52,0.00507058,0.00029662,0.00030430,-2.54166667
E,193.32,-0.00198787,0.005

In [234]:
coeffs = {'CC_di':[],
         'CC_eno3':[],
         'CC_eswa':[],
         'CC_wl':[],
         'CC_lower_wavelength_limit_for_spectra_fit':['217'],
         'CC_upper_wavelength_limit_for_spectra_fit':['240'],
         'CC_cal_temp':[]
         }

for line in data.splitlines():
    if 'T_CAL' in line:
        *ignore, cal_temp = line.split()
        coeffs['CC_cal_temp'].append(cal_temp)
    
    if line.startswith('E'):
        __, wl, eno3, eswa, __, di = line.split(',')
        coeffs['CC_di'].append(di)
        coeffs['CC_wl'].append(wl)
        coeffs['CC_eno3'].append(eno3)
        coeffs['CC_eswa'].append(eswa)
        
        
        

In [235]:
coeffs

{'CC_di': ['-12.66666667',
  '-10.58333333',
  '-10.62500000',
  '-8.29166667',
  '-2.54166667',
  '-3.91666667',
  '-0.20833333',
  '-2.12500000',
  '-3.04166667',
  '-3.37500000',
  '4.25000000',
  '58.08333333',
  '299.87500000',
  '1081.33333333',
  '2804.37500000',
  '5417.75000000',
  '8227.95833333',
  '10444.95833333',
  '11759.54166667',
  '12320.58333333',
  '12399.12500000',
  '12251.29166667',
  '12019.37500000',
  '11845.41666667',
  '11769.62500000',
  '11833.04166667',
  '12051.87500000',
  '12440.87500000',
  '13006.95833333',
  '13746.54166667',
  '14663.83333333',
  '15731.50000000',
  '16945.83333333',
  '18206.83333333',
  '19476.83333333',
  '20651.08333333',
  '21650.37500000',
  '22349.41666667',
  '22714.95833333',
  '22694.45833333',
  '22312.58333333',
  '21659.00000000',
  '20797.41666667',
  '19885.70833333',
  '18982.91666667',
  '18171.33333333',
  '17496.45833333',
  '17003.12500000',
  '16667.45833333',
  '16503.87500000',
  '16527.41666667',
  '16687.12

In [236]:
df = pd.DataFrame()
for i in coeffs.keys():
    df = df.append({
        'serial':'NTR-1090',
        'name':i,
        'value':coeffs.get(i),
        'notes':'stuff'
    }, ignore_index=True)

In [237]:
df

,name,notes,serial,value
0,CC_di,stuff,NTR-1090,"[-12.66666667, -10.58333333, -10.62500000, -8...."
1,CC_eno3,stuff,NTR-1090,"[0.00208249, 0.00278842, -0.00420238, 0.001800..."
2,CC_eswa,stuff,NTR-1090,"[0.00263663, 0.00220094, 0.00905437, 0.0015782..."
3,CC_wl,stuff,NTR-1090,"[189.32, 190.12, 190.92, 191.72, 192.52, 193.3..."
4,CC_lower_wavelength_limit_for_spectra_fit,stuff,NTR-1090,[217]
5,CC_upper_wavelength_limit_for_spectra_fit,stuff,NTR-1090,[240]
6,CC_cal_temp,stuff,NTR-1090,"[19.95, 19.95]"


In [240]:
fpath = '/home/andrew/Documents/OOI-CGSN/asset-management/calibration/NUTNRB/CGINS-NUTNRB-01090__20190401.csv'

In [241]:
df2 = pd.read_csv(fpath)

In [242]:
df

,name,notes,serial,value
0,CC_di,stuff,NTR-1090,"[-12.66666667, -10.58333333, -10.62500000, -8...."
1,CC_eno3,stuff,NTR-1090,"[0.00208249, 0.00278842, -0.00420238, 0.001800..."
2,CC_eswa,stuff,NTR-1090,"[0.00263663, 0.00220094, 0.00905437, 0.0015782..."
3,CC_wl,stuff,NTR-1090,"[189.32, 190.12, 190.92, 191.72, 192.52, 193.3..."
4,CC_lower_wavelength_limit_for_spectra_fit,stuff,NTR-1090,[217]
5,CC_upper_wavelength_limit_for_spectra_fit,stuff,NTR-1090,[240]
6,CC_cal_temp,stuff,NTR-1090,"[19.95, 19.95]"


In [243]:
df2

,serial,name,value,notes
0,NTR-1090,CC_cal_temp,19.95,date in filename comes from latest caldate wit...
1,NTR-1090,CC_di,"[-12.66666667, -10.58333333, -10.625, -8.29166...",NaN
2,NTR-1090,CC_eno3,"[0.00208249, 0.00278842, -0.00420238, 0.001800...",NaN
3,NTR-1090,CC_eswa,"[0.00263663, 0.00220094, 0.00905437, 0.0015782...",NaN
4,NTR-1090,CC_lower_wavelength_limit_for_spectra_fit,217,NaN
5,NTR-1090,CC_upper_wavelength_limit_for_spectra_fit,240,NaN
6,NTR-1090,CC_wl,"[189.32, 190.12, 190.92, 191.72, 192.52, 193.3...",NaN


In [271]:
name = 'CC_wl'

In [272]:
df[df['name']==name]['value']

3    [189.32, 190.12, 190.92, 191.72, 192.52, 193.3...
Name: value, dtype: object

In [273]:
df2[df2['name']==name]['value']

6    [189.32, 190.12, 190.92, 191.72, 192.52, 193.3...
Name: value, dtype: object

In [274]:
A = df[df['name']==name]['value']
A = [word.rstrip('0') for word in A[3]]

In [275]:
B = df2[df2['name']==name]['value']
B = B[6].replace('[','').replace(']','').replace(' ','').split(',')

In [276]:
A = [float(word) for word in A]

In [277]:
B = [float(word) for word in B]

In [278]:
A == B

True

In [279]:
np.isclose(A,B)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [280]:
B

[189.32,
 190.12,
 190.92,
 191.72,
 192.52,
 193.32,
 194.12,
 194.92,
 195.72,
 196.53,
 197.33,
 198.13,
 198.93,
 199.73,
 200.54,
 201.34,
 202.14,
 202.95,
 203.75,
 204.55,
 205.36,
 206.16,
 206.97,
 207.77,
 208.58,
 209.38,
 210.19,
 210.99,
 211.8,
 212.6,
 213.41,
 214.21,
 215.02,
 215.83,
 216.63,
 217.44,
 218.25,
 219.05,
 219.86,
 220.67,
 221.48,
 222.28,
 223.09,
 223.9,
 224.71,
 225.52,
 226.32,
 227.13,
 227.94,
 228.75,
 229.56,
 230.37,
 231.18,
 231.99,
 232.8,
 233.61,
 234.42,
 235.23,
 236.04,
 236.85,
 237.66,
 238.47,
 239.28,
 240.09,
 240.9,
 241.71,
 242.52,
 243.33,
 244.14,
 244.96,
 245.77,
 246.58,
 247.39,
 248.2,
 249.01,
 249.83,
 250.64,
 251.45,
 252.26,
 253.07,
 253.89,
 254.7,
 255.51,
 256.32,
 257.14,
 257.95,
 258.76,
 259.58,
 260.39,
 261.2,
 262.02,
 262.83,
 263.64,
 264.46,
 265.27,
 266.08,
 266.9,
 267.71,
 268.52,
 269.34,
 270.15,
 270.96,
 271.78,
 272.59,
 273.41,
 274.22,
 275.03,
 275.85,
 276.66,
 277.48,
 278.29,
 279.1,
 2

In [281]:
A

[189.32,
 190.12,
 190.92,
 191.72,
 192.52,
 193.32,
 194.12,
 194.92,
 195.72,
 196.53,
 197.33,
 198.13,
 198.93,
 199.73,
 200.54,
 201.34,
 202.14,
 202.95,
 203.75,
 204.55,
 205.36,
 206.16,
 206.97,
 207.77,
 208.58,
 209.38,
 210.19,
 210.99,
 211.8,
 212.6,
 213.41,
 214.21,
 215.02,
 215.83,
 216.63,
 217.44,
 218.25,
 219.05,
 219.86,
 220.67,
 221.48,
 222.28,
 223.09,
 223.9,
 224.71,
 225.52,
 226.32,
 227.13,
 227.94,
 228.75,
 229.56,
 230.37,
 231.18,
 231.99,
 232.8,
 233.61,
 234.42,
 235.23,
 236.04,
 236.85,
 237.66,
 238.47,
 239.28,
 240.09,
 240.9,
 241.71,
 242.52,
 243.33,
 244.14,
 244.96,
 245.77,
 246.58,
 247.39,
 248.2,
 249.01,
 249.83,
 250.64,
 251.45,
 252.26,
 253.07,
 253.89,
 254.7,
 255.51,
 256.32,
 257.14,
 257.95,
 258.76,
 259.58,
 260.39,
 261.2,
 262.02,
 262.83,
 263.64,
 264.46,
 265.27,
 266.08,
 266.9,
 267.71,
 268.52,
 269.34,
 270.15,
 270.96,
 271.78,
 272.59,
 273.41,
 274.22,
 275.03,
 275.85,
 276.66,
 277.48,
 278.29,
 279.1,
 2